In [1]:
import os
import numpy as np 
import pandas as pd

In [2]:
path = r"C:\Users\piush\Desktop\World Bank Poverty\submissions\c_sub"

all_files = os.listdir(path)

# Read and concatenate submissions
outs = [pd.read_csv(os.path.join(path, f), index_col=[0,1]) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "is_poor_" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
concat_sub.head()

,id,country,is_poor_0,is_poor_1,is_poor_2,is_poor_3,is_poor_4,is_poor_5,is_poor_6,is_poor_7,is_poor_8,is_poor_9,is_poor_10
0,6211,C,0.000122,0.000026,0.000110,0.000045,0.000633,0.000084,0.000080,0.000094,0.000461,0.000141,0.000223
1,77549,C,0.000122,0.000026,0.000020,0.000014,0.000086,0.000084,0.000059,0.000094,0.000040,0.000141,0.000127
2,34834,C,0.000028,0.000022,0.000020,0.000014,0.000036,0.000032,0.000023,0.000028,0.000040,0.000032,0.000009
3,74650,C,0.000028,0.000022,0.000020,0.000014,0.000036,0.000032,0.000023,0.000028,0.000040,0.000032,0.000102
4,65375,C,0.000145,0.000026,0.000199,0.000024,0.000417,0.001191,0.001674,0.000310,0.002492,0.000141,0.000369


In [3]:
concat_sub.iloc[:,2:15].head(2)

,is_poor_0,is_poor_1,is_poor_2,is_poor_3,is_poor_4,is_poor_5,is_poor_6,is_poor_7,is_poor_8,is_poor_9,is_poor_10
0,0.000122,0.000026,0.00011,0.000045,0.000633,0.000084,0.000080,0.000094,0.000461,0.000141,0.000223
1,0.000122,0.000026,0.00002,0.000014,0.000086,0.000084,0.000059,0.000094,0.000040,0.000141,0.000127


In [4]:
concat_sub['poor'] = concat_sub.iloc[:,2:15].mean(axis = 1)

In [5]:
c = concat_sub[['id','country','poor']]

In [6]:
c.to_csv("c_1.csv")

In [ ]:
# get the data fields ready for stacking
concat_sub['is_poor_max'] = concat_sub.iloc[:, 2:18].max(axis=1)
concat_sub['is_poor_min'] = concat_sub.iloc[:, 2:18].min(axis=1)
concat_sub['is_poor_mean'] = concat_sub.iloc[:, 2:18].mean(axis=1)
concat_sub['is_poor_median'] = concat_sub.iloc[:, 2:18].median(axis=1)

In [ ]:
# set up cutoff threshold for lower and upper bounds, easy to twist 
cutoff_lo = 0.8
cutoff_hi = 0.2

In [ ]:
concat_sub.head()

### Change the column names 

###### Mean Stacking

In [ ]:
concat_sub['is_poor'] = concat_sub['is_poor_mean']
concat_sub[['id','country', 'is_poor']].to_csv('stack_mean.csv', 
                                        index=False, float_format='%.6f')

###### Median Stacking

In [ ]:
concat_sub['is_poor'] = concat_sub['is_poor_median']
concat_sub[['id','country', 'is_poor']].to_csv('stack_median.csv', 
                                        index=False, float_format='%.6f')

###### PushOut + Median Stacking
Pushout strategy is a bit agressive given what it does...

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 1, 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             0, concat_sub['is_poor_median']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_pushout_median.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + Mean Stacking
MinMax seems more gentle and it outperforms the previous one given its peformance score.

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_mean']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_mean.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + Median Stacking

In [ ]:
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_median']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_median.csv', 
                                        index=False, float_format='%.6f')

###### MinMax + BestBase Stacking

In [ ]:
# load the model with best base performance
sub_base = pd.read_csv(r"C:\Users\pvaish10\Desktop\csv_poverty\submission_X_G_tr2.csv")

In [ ]:
concat_sub['is_poor_base'] = sub_base['poor']
concat_sub['is_poor'] = np.where(np.all(concat_sub.iloc[:,1:6] > cutoff_lo, axis=1), 
                                    concat_sub['is_poor_max'], 
                                    np.where(np.all(concat_sub.iloc[:,1:6] < cutoff_hi, axis=1),
                                             concat_sub['is_poor_min'], 
                                             concat_sub['is_poor_base']))
concat_sub[['id','country', 'is_poor']].to_csv('stack_minmax_bestbase.csv', 
                                        index=False, float_format='%.6f')